
## <font color='#3d59c6'>  INTRODUCCIÓN </font>

 **Estrategia**

* Extracción de Datos
* Transformación de Datos
* Carga de Datos
* Conclusiones


## <font color='#3d59c6'>  INSTALACIÓN DE PACKAGES </font>

In [ ]:
import pandas as pd
import numpy as np
import folium
import requests
import matplotlib.pyplot as plt
#Instalar plotly para visualizar los graficos
import plotly.express as px
import plotly.graph_objects as go
from pymongo import MongoClient
from io import BytesIO
import plotly.graph_objs as go
import plotly.offline as offline

ModuleNotFoundError: No module named 'pymongo'

## <font color='#3d59c6'>  EXTRACCIÓN DE DATOS </font>

In [ ]:
import requests
import pandas as pd

# URL del endpoint de la API
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson'

# Realizar la solicitud GET
response = requests.get(url)

# Verificar el estado de la respuesta
if response.status_code == 200:
    # Si la solicitud fue exitosa, obtener el contenido JSON de la respuesta
    data = response.json()

    # Crear un DataFrame de pandas con todas las variables disponibles en el JSON
    earthquakes = []
    for feature in data['features']:
        earthquake = feature['properties']
        earthquake['latitude'] = feature['geometry']['coordinates'][1]
        earthquake['longitude'] = feature['geometry']['coordinates'][0]
        earthquake['depth'] = feature['geometry']['coordinates'][2]  # Agregar profundidad
        earthquake['locationSource'] = earthquake['nst'] if 'nst' in earthquake else None  # Obtener la fuente de la ubicación
        earthquakes.append(earthquake)

    data = pd.DataFrame(earthquakes)

    print("Datos de terremotos guardados en el DataFrame 'datos'.")
else:
    # Si la solicitud no fue exitosa, imprimir el mensaje de error
    print("Error al realizar la solicitud:", response.status_code)
#Se limita los datos a cuya magnitud sea mayor o igual a 3
data = data[data['mag'] >= 3.0]

Datos de terremotos guardados en el DataFrame 'datos'.


In [ ]:
data[:10]

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,dmin,rms,gap,magType,type,title,latitude,longitude,depth,locationSource
14,3.54,"17 km WSW of Brinnon, Washington",1708101018830,1708110768990,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,72.0,3.6,2.341,...,0.05931,0.37,20.0,ml,earthquake,"M 3.5 - 17 km WSW of Brinnon, Washington",47.5950,-123.093833,50.370,93.0
19,4.40,"9 km E of San Felipe, Chile",1708098657575,1708100342040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,0.06800,0.59,72.0,mb,earthquake,"M 4.4 - 9 km E of San Felipe, Chile",-32.7419,-70.627500,88.931,37.0
21,4.40,"156 km SSE of Damusi, China",1708097907867,1708099621040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,4.33100,0.55,86.0,mb,earthquake,"M 4.4 - 156 km SSE of Damusi, China",36.4580,77.297000,133.350,63.0
23,3.00,"274 km ESE of Chiniak, Alaska",1708097454146,1708109984646,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,NaN,0.49,NaN,ml,earthquake,"M 3.0 - 274 km ESE of Chiniak, Alaska",56.4982,-148.245300,3.300,NaN
27,4.80,"210 km ESE of Onagawa Chō, Japan",1708095589232,1708096591040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,3.56900,0.91,139.0,mb,earthquake,"M 4.8 - 210 km ESE of Onagawa Chō, Japan",37.8787,143.736000,10.000,58.0
32,4.90,"256 km W of Tual, Indonesia",1708093413839,1708094429040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,1.62200,0.58,122.0,mb,earthquake,"M 4.9 - 256 km W of Tual, Indonesia",-6.0505,130.476800,116.767,57.0
35,4.50,"234 km NW of Houma, Tonga",1708092042357,1708095599040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,2.18000,0.66,76.0,mb,earthquake,"M 4.5 - 234 km NW of Houma, Tonga",-19.7566,-176.980200,325.526,43.0
36,4.40,"68 km NNW of Murghob, Tajikistan",1708091236544,1708099254732,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,1.0,2.0,NaN,...,1.56200,0.60,106.0,mb,earthquake,"M 4.4 - 68 km NNW of Murghob, Tajikistan",38.6865,73.530900,94.042,25.0
68,5.10,"Izu Islands, Japan region",1708081262731,1708082398040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,1.76200,0.52,130.0,mb,earthquake,"M 5.1 - Izu Islands, Japan region",31.4812,140.604900,52.679,51.0
79,4.50,"3 km ESE of Shiraoka, Japan",1708077133768,1708080042333,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,1.0,1.0,NaN,...,1.31500,0.56,125.0,mb,earthquake,"M 4.5 - 3 km ESE of Shiraoka, Japan",36.0046,139.695500,61.377,51.0


## <font color='#3d59c6'>  TRANSFORMACIÓN DE DATOS </font>

### <font color='#3d59c6'>  VISUALIZACIONES DE TERREMOTOS </font>

### Actividad sísmica mundiañl ocurrida entre el 10 de enero de 2024 y el 9 de febrero de 2024.

Para entender los patrones y la distribución de los terremotos, es esencial presentar la información de una manera que sea tanto informativa como accesible. Para lograr esto, hemos recurrido a una fuente confiable y autorizada: el Servicio Geológico de Estados Unidos (USGS).

Los datos que vamos a presentar provienen de la API de terremotos del USGS, específicamente la consulta que nos proporciona información sobre eventos sísmicos de magnitud igual o superior a 3 ocurrida entre el 10 de enero de 2024 y el 9 de febrero de 2024. Estos datos han sido convertidos a formato CSV para facilitar su manipulación y análisis en nuestro entorno de trabajo.

Ahora, nos adentraremos en la visualización de estos datos a través de gráficos interactivos creados con la biblioteca de Plotly en Python. Plotly nos permite transformar números y coordenadas en representaciones visuales que destacan la frecuencia, magnitud y ubicación de los sismos. Veremos:

1. **Mapas de Dispersión Geográfica**: Estos nos mostrarán la ubicación exacta de los terremotos, con puntos codificados por colores y tamaños que representan la magnitud de cada evento. Este tipo de gráfico nos ayuda a identificar rápidamente áreas de alta actividad sísmica.

2. **Histogramas y Gráficos de Barras**: Para comprender mejor la distribución de las magnitudes de terremotos y su frecuencia, utilizaremos gráficos que nos permitan visualizar la cantidad de terremotos en diferentes rangos de magnitud.

3. **Gráficos de Línea de Tiempo**: A través de estos, podremos observar la secuencia de los terremotos a lo largo del tiempo, proporcionando una perspectiva clara de la actividad sísmica a lo largo de un periodo específico.

Cada visualización está diseñada para aportar una capa de comprensión, permitiéndonos descifrar patrones y extraer conclusiones sobre la actividad sísmica global. Con la interactividad de nuestras gráficas, invitamos a los espectadores a sumergirse en los datos, explorando cada terremoto con la riqueza de detalle que solo la visualización de datos puede proporcionar.ctiva y enriquecedora.

## Terremotos en todo el mundo.

Este gráfico interactivo proporciona una representación visual inmediata del registro global de terremotos con magnitudes de 3.0 o superiores. Cada círculo en el mapa representa un terremoto, con su ubicación geográfica precisa dada por las coordenadas de latitud y longitud. La escala de colores, que va de amarillo a morado oscuro, se utiliza para indicar la magnitud de cada terremoto, con tonos más oscuros que representan magnitudes más altas.

In [ ]:
fig = px.scatter_geo(data,
                     lat='latitude',
                     lon='longitude',
                     color='mag',  # Color de los puntos basado en la magnitud
                     hover_name='place',  # Mostrar el lugar al pasar el ratón por encima
                     size='mag',  # Tamaño de los puntos basado en la magnitud
                     projection='natural earth',  # Tipo de proyección del mapa
                     title='Terremotos en todo el mundo, en el último mes.')

# Configurar las dimensiones del layout del gráfico
fig.update_layout(width=1000, height=600)  # Puedes ajustar estos valores según tus necesidades

# Convertir la figura a bytes
img_reg_globales = fig.to_image(format="png")

fig.show()

# Guardar el gráfico como un archivo HTML
offline.plot(fig, filename='Grafico_terremoto_ultimo_mes.html', auto_open=False)


Al observar el mapa, se pueden destacar varios puntos clave:

- Concentración de Terremotos: Los círculos están densamente agrupados a lo largo de ciertas áreas, destacando las zonas de alta actividad sísmica conocidas como las fronteras de placas tectónicas.

- Magnitud de los Terremotos: Los tamaños de los círculos varían, lo que indica la magnitud de los terremotos. Los terremotos más grandes, con magnitudes más cercanas a 7, están marcados con círculos de color más oscuro. Esto nos permite identificar rápidamente las regiones que han experimentado terremotos más potentes.

- Distribución Geográfica: Aunque hay una presencia global de terremotos, es evidente que hay regiones con una actividad sísmica significativamente menor, como es Europa, donde España es una privilegiada al escapar de la actividad sismica. Podemos ver también que las áreas centrales de los continentes tienden a tener menos terremotos que las regiones costeras o las islas.islas.

## Terremotos sobre grado 6.


In [ ]:
data_filtrado = data[data['mag'] >= 6.0]

fig = px.scatter_geo(data_filtrado,
                     lat='latitude',
                     lon='longitude',
                     color='mag',
                     hover_name='place',
                     size='mag',
                     projection='natural earth',
                     title='Terremotos significativos en todo el mundo (>6.0 de magnitud)')

fig.update_layout(width=1000, height=600)

# Convertir la figura a bytes
img_terremotos_grado_6 = fig.to_image(format="png")

fig.show()

# Guardar el gráfico como un archivo HTML
offline.plot(fig, filename='Grafico_terremoto_todo_mundo.html', auto_open=False)


- Notables Eventos Sísmicos: El mapa destaca terremotos significativos en Brasil y China, poniendo de relieve que la actividad sísmica no se limita solo a las zonas tradicionalmente activas como el Cinturón de Fuego del Pacífico.

In [ ]:
fig = px.histogram(data, x='mag', nbins=50, title='Distribución de Magnitudes de Terremotos')
fig.update_layout(width=1000, height=600)

# Convertir la figura a bytes
img_hist_terremotos = fig.to_image(format="png")


fig.show()

# Guardar el gráfico como un archivo HTML
offline.plot(fig, filename='Grafico_Distribucion_terremoto.html', auto_open=False)


En el histograma se evidencia que la mayor concentración de terremotos se encuentra en el intervalo de magnitud entre 4 y 5 grados.

In [ ]:
fig = px.scatter(data, x='depth', y='mag', title='Profundidad vs Magnitud de Terremotos',
                 labels={'depth':'Profundidad (km)', 'mag':'Magnitud'})

# Convertir la figura a bytes
img_profundidad_magnitud = fig.to_image(format="png")

fig.show()

# Guardar el gráfico como un archivo HTML
offline.plot(fig, filename='Grafico_Profundidad_magnitud.html', auto_open=False)


**Concentración de Datos:** Hay una gran concentración de datos en el rango de profundidad de 0 a 100 kilómetros y con magnitudes entre aproximadamente 3 y 6. Esto sugiere que la mayoría de los terremotos registrados en este conjunto de datos son relativamente superficiales y tienen magnitudes moderadas.

**Correlación:** No parece haber una correlación clara o lineal entre la profundidad y la magnitud de los terremotos. Los puntos están bastante dispersos sin un patrón claro que indique que una variable aumenta o disminuye consistentemente con la otra.

## Comparacion placas tecntonicas y terremotos del último mes.


<img src="https://significado.com/img/cien/placas-tectonicas-distribucion-planisferio.jpg" width="1000" height="600" alt="Distribución de las Placas Tectónicas">




En el análisis del mapa de calor de la actividad sísmica del último mes (Mapa que esta abajo de este texto) y su comparación con la distribución global de las placas tectónicas, es evidente que hay una correlación significativa entre los patrones sísmicos y las ubicaciones de las placas tectónicas. Los movimientos y colisiones de estas placas suelen ser el origen de los terremotos, lo cual se refleja claramente en el gráfico, demostrando que los terremotos tienden a concentrarse en las áreas donde estas placas se encuentran o se deslizan unas contra otras.

In [ ]:
fig = go.Figure(go.Densitymapbox(lat=data['latitude'], lon=data['longitude'], z=data['mag'], radius=10))
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(width=1000, height=600)

# Convertir la figura a bytes
img_mapa_calor = fig.to_image(format="png")

fig.show()

# Guardar el gráfico como un archivo HTML
offline.plot(fig, filename='Grafico_calor.html', auto_open=False)


In [ ]:
data['time'] = pd.to_datetime(data['time'])

fig = px.line(data, x='time', y='mag', title='Magnitud de Terremotos a lo Largo del Tiempo',
              labels={'time':'Tiempo', 'mag':'Magnitud'})

# Convertir la figura a bytes
img_magnitud_terremotos = fig.to_image(format="png")

fig.show()

# Guardar el gráfico como un archivo HTML
offline.plot(fig, filename='Grafico_Magnitud_terremoto.html', auto_open=False)



**Ausencia de patrones claros:** No parece haber un patrón claro o cíclico en la ocurrencia de terremotos de alta magnitud, lo que indica que estos eventos no siguen una tendencia predecible en este conjunto de datos.

**Rango de magnitud:** Las magnitudes parecen oscilar principalmente entre aproximadamente 3 y 7, con la mayoría de los terremotos presentando magnitudes en el extremo inferior de ese rango. Los eventos de magnitud 6 o más parecen ser menos frecuentes.

### <font color='#3d59c6'>  TABLA DE FRECUENCIAS: Centros Observatorios de Terremotos </font>

La API nos da información del lugar en el que se realizaron las observaciones del terremoto. Esta
información fue almacenada en una tabla de frecuencia para poder identificar cuáles han sido los
centros observatorios con mayor actividad en el último mes. Esta tabla también tiene los datos de
la magnitud media de los terremotos captados en cada centro de observación.

In [ ]:

def frequency_table(data):
    # Calcular la frecuencia de cada zona
    frequency = data['locationSource'].value_counts()

    # Calcular la media de la magnitud para cada zona
    magnitud_media = data.groupby('locationSource')['mag'].mean()

    # Ordenar las zonas por frecuencia descendente
    frequency = frequency.sort_values(ascending=False)

    # Crear el DataFrame con la frecuencia y la magnitud media
    frequency_table = pd.DataFrame({'Zona': frequency.index, 'Frecuencia': frequency.values})

    # Agregar la columna de magnitud media al DataFrame
    frequency_table['Magnitud Media'] = frequency_table['Zona'].map(magnitud_media)

    return frequency_table

# Ejemplo de uso
frequency_table(data)
# En este ademas ponemos la media de la magnitud

La API tiene un campo llamado "place", donde hace una descripción del sitio y el país en donde
ocurrió el terremoto. Sin embargo, luego de analizar los datos, se observa que el país en muchas
ocasiones no está correcto. El caso más común y recurrente, es el de Estados Unidos, en donde en vez del país aparece el nombre del estado.
Para poder identificar de manera precisa el país de origen del movimiento telúrico, se han utilizado
los campos de latitud y longitud. Usando la librería ’reverse geocode’, se toman las coordenadas
geográficas latitud y longitud y se convierten para obtener el país en donde se encuentran

### <font color='#3d59c6'>  TABLA DE FRECUENCIAS: Per Country </font>

In [ ]:
import reverse_geocode

# Función para obtener el país a partir de las coordenadas
def get_country(row):
    coordinates = (row['latitude'], row['longitude'])
    location = reverse_geocode.search([coordinates])
    return location[0]['country']

# Agrega una nueva columna 'country' al DataFrame
data['country'] = data.apply(get_country, axis=1)



Una vez agregada esta nueva columna de ’country’ al dataset, se procede a crear una tabla de frecuencias que identifique, esta vez, aquellos países que han tenido mayor cantidad de terremotos en
el último mes y la magnitud promedio de los mismos.
Se crea así un histograma con los 15 paises de mayor actividad sísmica en el último mes.


In [ ]:

def country_frequency_table(data):
    # Calcular la frecuencia de cada país
    frequency = data['country'].value_counts()

    # Tomar solo los primeros "X" países en términos de frecuencia
    top_countries = frequency.head(15).index

    # Filtrar el DataFrame original para incluir solo las filas correspondientes a los primeros países
    filtered_data = data[data['country'].isin(top_countries)]

    # Calcular la media de la magnitud para cada país
    magnitud_media = filtered_data.groupby('country')['mag'].mean()

    # Ordenar los países por frecuencia descendente
    frequency = frequency.sort_values(ascending=False)

    # Crear el DataFrame con la frecuencia y la magnitud media
    frequency_table = pd.DataFrame({'País': top_countries, 'Frecuencia': frequency[top_countries].values})

    # Agregar la columna de magnitud media al DataFrame
    frequency_table['Magnitud Media'] = frequency_table['País'].map(magnitud_media)

    return frequency_table

# Ejemplo de uso
country_frequency_table(data)


In [ ]:
import matplotlib.pyplot as plt
import base64
import io

def country_frequency_histogram(data):
    # Calcular la frecuencia de cada país
    frequency = data['country'].value_counts()

    # Tomar solo los primeros 15 países en términos de frecuencia
    top_countries = frequency.head(15)

    # Crear el histograma
    plt.figure(figsize=(10, 6))
    top_countries.plot(kind='bar')
    plt.xlabel('País')
    plt.ylabel('Frecuencia')
    plt.title('Frecuencia de Terremotos por País (Top 15)')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()


 # Convertir la figura a un formato que se pueda almacenar en MongoDB
buffer = io.BytesIO()
plt.savefig(buffer, format='png')
buffer.seek(0)
image_png = buffer.getvalue()
buffer.close()

# Codificar la imagen en base64
img_frecuencia_terremoto = base64.b64encode(image_png).decode('utf-8')

# Ejemplo de uso
country_frequency_histogram(data)

# Guardar el gráfico como un archivo HTML
offline.plot(img_frecuencia_terremoto, filename='Grafico_Frecuencia_terremotos.html', auto_open=False)


## <font color='#3d59c6'>  CARGA DE DATOS </font>

In [ ]:
# Conectar a la base de datos MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['bd_terremotos']
collection = db['terremotos']


In [ ]:
# Crear una lista de los documentos que contienen las imágenes
imagenes_para_insertar = [
    {'image': img_reg_globales},
    {'image': img_terremotos_grado_6},
    {'image': img_hist_terremotos},
    {'image': img_profundidad_magnitud},
    {'image': img_mapa_calor},
    {'image': img_magnitud_terremotos},
    {'image': img_frecuencia_terremoto}
]

# Insertar las imágenes en la base de datos
resultados_insercion = collection.insert_many(imagenes_para_insertar)

# Verificar si la inserción fue exitosa para ambas imágenes
if resultados_insercion.acknowledged:
    print("Imágenes insertadas correctamente en la base de datos.")
else:
    print("Error al insertar las imágenes en la base de datos.")


## <font color='#3d59c6'>  CONCLUSIONES </font>

Siguiendo lo aprendido en la clase de Estructura de Datos y su Almacenamiento, hemos optado por utilizar una base de datos NoSQL, en este caso MongoDB, para almacenar la información de forma local. Esta decisión nos permite mantener un repositorio actualizado cada vez que realicemos cambios o actualizaciones en nuestros datos.

En nuestro proyecto, hemos almacenado las imágenes que fueron otorgadas durante el análisis y la transformación de los datos. Estas imágenes representan información relevante para nuestro estudio y son un elemento fundamental en nuestro proceso de análisis de datos.

Al utilizar MongoDB como nuestro sistema de gestión de base de datos, tenemos la flexibilidad necesaria para almacenar datos no estructurados, como imágenes, de manera eficiente y escalable. Esto nos permite organizar y acceder a nuestra información de manera efectiva, facilitando así el desarrollo de nuestro proyecto y la realización de análisis posteriores.

Además, al mantener la información localmente, tenemos un mayor control sobre nuestros datos y podemos trabajar en un entorno familiar y seguro. Esto nos brinda la confianza necesaria para realizar experimentos y pruebas sin preocuparnos por la integridad o la disponibilidad de nuestros datos.

Hemos elegido MongoDB como nuestra solución de almacenamiento de datos debido a su capacidad para manejar datos no estructurados, su escalabilidad y su facilidad de uso. Al mantener nuestras imágenes en una base de datos local, podemos asegurar un acceso rápido y seguro a nuestra información, lo que nos permite avanzar en nuestro proyecto de manera eficiente y efectiva.